# Gesture Recognition - Model 7
#### Conv3D Model with comprehensive preprocessing (using OpenCV library - Resize and Data augmetation with Affine transformation, Horizontal Flip, Cropping and Resizing,  Usage of Optimizer, ReduceLROnPlateau to reduce learning rate and experimenting with various sizes of batch sizes and Epochs)
#### Added Batch Normalization over Model 6
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. We will set the initial `batch_size` here. Note that we will experimentally set the batch size in such a way that you are able to use the GPU in full capacity. Keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

***Note*** in our implementaiton true **batch_size** is always 2 fold as we agument our data. if if batch size is 10, true batch size at the run time is 20, as twice the number of data set loaded are added to the batch


## Generator
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [4]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # lenght of the video frames
nb_channel = 3 # number of channels in images 3 for color(RGB) and 1 for Gray

In [5]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [7]:
# Function that initializes all the batch image data and labels
def initialize_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [8]:
def process_batch_images(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = initialize_batch_data(batch_size)
    
    # For building an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = initialize_batch_data(batch_size)
    
    # For building an augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = initialize_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generating a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [9]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)       
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model 7

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

#nb_filters = [8,16,32,64]
#nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model

model = Sequential()

model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [16]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 198s 3s/step - loss: 1.8549 - categorical_accuracy: 0.2386 - val_loss: 1.5631 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2020-03-3012_14_57.409404/model-00001-1.85619-0.23881-1.56313-0.37000.h5
Epoch 2/20
67/67 [==============================] - 31s 465ms/step - loss: 1.5514 - categorical_accuracy: 0.2952 - val_loss: 1.4909 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2020-03-3012_14_57.409404/model-00002-1.55143-0.29519-1.49089-0.23000.h5
Epoch 3/20
67/67 [==============================] - 34s 502ms/step - loss: 1.5096 - categorical_accuracy: 0.3068 - val_loss: 1.4682 - val_categorical_accuracy: 0.4200

Epoch 00003: saving model to model_init_2020-03-3012_14_57.409404/model-00003-1.50962-0.30680-1.46817-0.42000.h5
Epoch 4/20
67/67 [==============================] - 3

In [17]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 17s 491ms/step - loss: 0.6731 - categorical_accuracy: 0.7516 - val_loss: 1.1284 - val_categorical_accuracy: 0.5200

Epoch 00001: saving model to model_init_2020-03-3012_14_57.409404/model-00001-0.67306-0.75163-1.12838-0.52000.h5
Epoch 2/20
34/34 [==============================] - 14s 401ms/step - loss: 0.6840 - categorical_accuracy: 0.7418 - val_loss: 1.0012 - val_categorical_accuracy: 0.5800

Epoch 00002: saving model to model_init_2020-03-3012_14_57.409404/model-00002-0.68402-0.74183-1.00117-0.58000.h5
Epoch 3/20
34/34 [==============================] - 15s 428ms/step - loss: 0.5700 - categorical_accuracy: 0.7843 - val_loss: 1.1143 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2020-03-3012_14_57.409404/model-00003-0.57005-0.78431-1.11427-0.52000.h5
Epoch 4/20
34/34 [==============================] - 14s 403ms/step - loss: 0.5641 - categorical_accuracy: 0.7876 - val_loss: 1.1260 - val_categorical_accurac

In [18]:
batch_size = 30
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 11s 484ms/step - loss: 0.7473 - categorical_accuracy: 0.7005 - val_loss: 1.2524 - val_categorical_accuracy: 0.3750

Epoch 00001: saving model to model_init_2020-03-3012_14_57.409404/model-00001-0.74725-0.70048-1.25240-0.37500.h5
Epoch 2/30
23/23 [==============================] - 10s 420ms/step - loss: 0.6093 - categorical_accuracy: 0.7585 - val_loss: 0.9343 - val_categorical_accuracy: 0.6250

Epoch 00002: saving model to model_init_2020-03-3012_14_57.409404/model-00002-0.60928-0.75845-0.93429-0.62500.h5
Epoch 3/30
23/23 [==============================] - 10s 443ms/step - loss: 0.8575 - categorical_accuracy: 0.6763 - val_loss: 1.2144 - val_categorical_accuracy: 0.4750

Epoch 00003: saving model to model_init_2020-03-3012_14_57.409404/model-00003-0.85745-0.67633-1.21444-0.47500.h5
Epoch 4/30
23/23 [==============================] - 9s 412ms/step - loss: 0.5874 - categorical_accuracy: 0.8164 - val_loss: 1.0338 - val_categorical_accuracy

23/23 [==============================] - 10s 429ms/step - loss: 0.6120 - categorical_accuracy: 0.7778 - val_loss: 0.9834 - val_categorical_accuracy: 0.5500

Epoch 00027: saving model to model_init_2020-03-3012_14_57.409404/model-00027-0.61196-0.77778-0.98338-0.55000.h5
Epoch 28/30
23/23 [==============================] - 10s 437ms/step - loss: 0.5029 - categorical_accuracy: 0.8068 - val_loss: 1.1945 - val_categorical_accuracy: 0.4750

Epoch 00028: saving model to model_init_2020-03-3012_14_57.409404/model-00028-0.50289-0.80676-1.19450-0.47500.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 7.450580950807417e-12.
Epoch 29/30
23/23 [==============================] - 9s 394ms/step - loss: 0.6683 - categorical_accuracy: 0.7150 - val_loss: 1.0925 - val_categorical_accuracy: 0.4750

Epoch 00029: saving model to model_init_2020-03-3012_14_57.409404/model-00029-0.66826-0.71498-1.09245-0.47500.h5
Epoch 30/30
23/23 [==============================] - 9s 412ms/step - loss: 0.8148 - ca

In [19]:
batch_size = 50
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
14/14 [==============================] - 7s 508ms/step - loss: 0.6105 - categorical_accuracy: 0.7619 - val_loss: 0.9946 - val_categorical_accuracy: 0.5000

Epoch 00001: saving model to model_init_2020-03-3012_14_57.409404/model-00001-0.61051-0.76190-0.99462-0.50000.h5
Epoch 2/30
14/14 [==============================] - 6s 441ms/step - loss: 0.7467 - categorical_accuracy: 0.7698 - val_loss: 1.3532 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2020-03-3012_14_57.409404/model-00002-0.74668-0.76984-1.35323-0.30000.h5
Epoch 3/30
14/14 [==============================] - 6s 404ms/step - loss: 0.7932 - categorical_accuracy: 0.7222 - val_loss: 1.0622 - val_categorical_accuracy: 0.6500

Epoch 00003: saving model to model_init_2020-03-3012_14_57.409404/model-00003-0.79318-0.72222-1.06220-0.65000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.8626452377018543e-12.
Epoch 4/30
14/14 [==============================] - 6s 409ms/step - loss: 0.54

In [20]:
batch_size = 90
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - 4s 520ms/step - loss: 0.5153 - categorical_accuracy: 0.8194 - val_loss: 1.0181 - val_categorical_accuracy: 0.5500

Epoch 00001: saving model to model_init_2020-03-3012_14_57.409404/model-00001-0.51534-0.81944-1.01806-0.55000.h5
Epoch 2/30
8/8 [==============================] - 4s 450ms/step - loss: 0.8896 - categorical_accuracy: 0.6528 - val_loss: 1.2221 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2020-03-3012_14_57.409404/model-00002-0.88965-0.65278-1.22207-0.40000.h5
Epoch 3/30
8/8 [==============================] - 4s 478ms/step - loss: 0.4511 - categorical_accuracy: 0.9028 - val_loss: 0.9748 - val_categorical_accuracy: 0.6000

Epoch 00003: saving model to model_init_2020-03-3012_14_57.409404/model-00003-0.45110-0.90278-0.97477-0.60000.h5
Epoch 4/30
8/8 [==============================] - 4s 459ms/step - loss: 0.5209 - categorical_accuracy: 0.8056 - val_loss: 1.0594 - val_categorical_accuracy: 0.4500

E


Epoch 00027: saving model to model_init_2020-03-3012_14_57.409404/model-00027-0.91160-0.63889-1.27551-0.40000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 4.440892309431682e-19.
Epoch 28/30
8/8 [==============================] - 4s 467ms/step - loss: 0.8259 - categorical_accuracy: 0.6250 - val_loss: 0.9894 - val_categorical_accuracy: 0.6500

Epoch 00028: saving model to model_init_2020-03-3012_14_57.409404/model-00028-0.82594-0.62500-0.98937-0.65000.h5
Epoch 29/30
8/8 [==============================] - 3s 412ms/step - loss: 0.5972 - categorical_accuracy: 0.7361 - val_loss: 1.0583 - val_categorical_accuracy: 0.5500

Epoch 00029: saving model to model_init_2020-03-3012_14_57.409404/model-00029-0.59723-0.73611-1.05830-0.55000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 2.220446154715841e-19.
Epoch 30/30
8/8 [==============================] - 4s 438ms/step - loss: 0.6607 - categorical_accuracy: 0.7917 - val_loss: 1.0306 - val_categorical_accuracy: 0.6500



In [21]:
batch_size = 90
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
8/8 [==============================] - 4s 501ms/step - loss: 0.4907 - categorical_accuracy: 0.7500 - val_loss: 1.1189 - val_categorical_accuracy: 0.6000

Epoch 00001: saving model to model_init_2020-03-3012_14_57.409404/model-00001-0.49065-0.75000-1.11885-0.60000.h5
Epoch 2/20
8/8 [==============================] - 3s 435ms/step - loss: 0.6201 - categorical_accuracy: 0.7083 - val_loss: 1.1941 - val_categorical_accuracy: 0.4500

Epoch 00002: saving model to model_init_2020-03-3012_14_57.409404/model-00002-0.62009-0.70833-1.19412-0.45000.h5
Epoch 3/20
8/8 [==============================] - 4s 445ms/step - loss: 0.7236 - categorical_accuracy: 0.7222 - val_loss: 0.9403 - val_categorical_accuracy: 0.6000

Epoch 00003: saving model to model_init_2020-03-3012_14_57.409404/model-00003-0.72362-0.72222-0.94031-0.60000.h5
Epoch 4/20
8/8 [==============================] - 3s 419ms/step - loss: 0.7370 - categorical_accuracy: 0.6389 - val_loss: 0.9886 - val_categorical_accuracy: 0.6000

E